In [1]:
# bitfit微调方法


import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


cache_dir = "/root/autodl-tmp"




In [2]:


from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from torch.optim import Adam
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import torch
from datasets import Dataset




In [3]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")

tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh", cache_dir=os.path.join(cache_dir,"bloom-1b4-zh"))

ds[0]

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

{'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
 'input': '',
 'instruction': '保持健康的三个提示。'}

In [4]:

def process_func(example):
    max_length = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"].strip(), "\nAssistant: "]))
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [5]:

tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds


Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:

model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True,cache_dir=os.path.join(cache_dir, "bloom-1b4-zh"))


sum(param.numel() for param in model.parameters())


1303111680

In [10]:

# bitfit 
# 选择模型参数里所有的bias部分


num_param = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()

print(num_param)   # 用于微调的参数数量



544768


In [12]:

num_param / sum(param.numel() for param in model.parameters())



0.000418051659240749

In [15]:


args = TrainingArguments(output_dir="/root/autodl-tmp/bitfit_finetuned_model",
                        per_device_train_batch_size=16,
                        gradient_accumulation_steps=4,
                        logging_steps=10,
                        num_train_epochs=1)


trainer = Trainer(model=model, args = args, data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
                 train_dataset=tokenized_ds)



trainer.train()


Step,Training Loss
10,2.315700
20,2.287900
30,2.321900
40,2.324500
50,2.311300
60,2.296300
70,2.352200
80,2.343200
90,2.302000
100,2.343800


TrainOutput(global_step=419, training_loss=2.301802710303259, metrics={'train_runtime': 812.9502, 'train_samples_per_second': 33.038, 'train_steps_per_second': 0.515, 'total_flos': 2.950268950413312e+16, 'train_loss': 2.301802710303259, 'epoch': 0.9982132221560452})

In [19]:



print(model.device)


model = model.cuda()

ipt = tokenizer("Human: {}\n {}".format("你是一名考试技巧丰富的学生，按照学生各个维度进行总结考试技巧", "").strip() + "\nAssistant:", return_tensors="pt").to(model.device)

print(tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0],skip_special_tokens=True))




cuda:0
Human: 你是一名考试技巧丰富的学生，按照学生各个维度进行总结考试技巧
Assistant: 是的，我一直都很努力地准备考试并严格要求自己。在接下来的考试中，我将从不同的角度进行总结和思考，以增强自己的考试技巧。
1. 阅读题型
阅读题型是考试中最常见的，也是难度最高的题型。因此，我将从这些方面来总结阅读：
1.1 题目选项的选择
阅读时，题目中通常会给出一些相关信息，考生需要在这些信息的基础上进行选项的选择，以保证能正确阅读和分析整体信息，找到正确答案。


In [17]:
model.generate??

Signature:
model.generate(
    inputs: Optional[torch.Tensor] = None,
    generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None,
    logits_processor: Optional[transformers.generation.logits_process.LogitsProcessorList] = None,
    stopping_criteria: Optional[transformers.generation.stopping_criteria.StoppingCriteriaList] = None,
    prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
    synced_gpus: Optional[bool] = None,
    assistant_model: Optional[ForwardRef('PreTrainedModel')] = None,
    streamer: Optional[ForwardRef('BaseStreamer')] = None,
    negative_prompt_ids: Optional[torch.Tensor] = None,
    negative_prompt_attention_mask: Optional[torch.Tensor] = None,
    **kwargs,
) -> Union[transformers.generation.utils.GenerateDecoderOnlyOutput, transformers.generation.utils.GenerateEncoderDecoderOutput, transformers.generation.utils.GenerateBeamDecoderOnlyOutput, transformers.generation.utils.GenerateBeam